In [1]:
import numpy as np
import pandas as pd
import random, pickle, argparse, json, os, urllib2
from collections import OrderedDict
from operator import itemgetter
import json

In [28]:
def query_from(q, f):
    q = q+'&from='+str(f)
    response = urllib2.urlopen(q)
    data = json.load(response)
    subject_ids = np.array(range(len(data['hits'])), dtype=np.object)
    for (i, hit) in enumerate(data['hits']):
        subject_ids[i] = hit['id']
    return subject_ids

def query(q):
    response = urllib2.urlopen(q)
    data = json.load(response)
    nb_requests = 1 + data['total'] / 1000
    if nb_requests > 10: # maximum number of pages due to API pagination restrection
        nb_requests = 10
    subject_ids = query_from(q, 0)
    for i in range(nb_requests)[1:]:
        f = i * 1000
        next_request = query_from(q, f)
        subject_ids = np.hstack((subject_ids, next_request))
    return subject_ids.tolist()

In [29]:
def find_intersection(list_a, list_b):
    return list(set(list_a) & set(list_b))
a = ['hussein', 'tayseer', 'al-natsheh']
b = ['loay', 'hussein', 'al-natsheh']
find_intersection(a, b)

['al-natsheh', 'hussein']

In [30]:
inv_index = json.load(open('../RecSys_Exp_files/182_381_vec150_results/output_paragraph_inversed_index.json','rb'))
print 'original inversed_index'
print inv_index.items()[:3]
inversed_index = dict()
for (k, v) in inv_index.items():
    key = k.split('_')[1]
    inversed_index[key] = v
print 'processed inversed_index'
print inversed_index.items()[:3]

original inversed_index
[(u'ISTEX_D89FA3AC3521074D46F4245762153DF497BFFA1F', 2002320), (u'ISTEX_18EAF4D6A126B077EB38667801D1B7292F32FF49', 2483732), (u'ISTEX_5F91044435FCC4FABB9F02E31467DCFE75F4A7BE', 1429049)]
processed inversed_index
[(u'FCF1393F9B8136AC08FB67E88F94F3CF62C17288', 3517138), (u'482E1102A1114327A744FD2ADB4D9F8FF7E9A70B', 751643), (u'A81022B6295AE66F68A10222C3B94A06B033C1BA', 3983232)]


In [32]:
def topic_get_input(topic, categories=0):
    if not categories:
        initial_corpus = query('https://api.istex.fr/document/?q=((title:%22'+topic+'%22%20OR%20abstract:%22'+topic+'%22)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id')
        _gs = query('https://api.istex.fr/document/?q=((subject.value:%22'+topic+'%22%20OR%20keywords:%22'+topic+'%22%20)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id')
    else:
        initial_corpus = query('https://api.istex.fr/document/?q=((title:%22'+topic+'%22%20OR%20subject.value:%22'+topic+'%22%20OR%20keywords:%22'+topic+'%22%20OR%20abstract:%22'+topic+'%22)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id')
        _gs = query('https://api.istex.fr/document/?q=((categories.wos:%22'+topic+'%22%20AND%20corpusName:%22elsevier%22)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id')
    initial_corpus = find_intersection(initial_corpus, inversed_index.keys())
    test_set = {x for x in _gs if x not in initial_corpus}
    test_set = find_intersection(test_set, inversed_index.keys())
    initial_corpus = list(initial_corpus)
    test = list(test_set)
    return initial_corpus, test

In [33]:
initial_corpus, test = topic_get_input('Biodiversity%20Conservation')
print 'size of initial corpus:', len(initial_corpus)
print 'size of ground_truth:', len(test)

size of initial corpus: 344
size of ground_truth: 55


In [44]:
queries =[]
for istex_id in initial_corpus:
    q = 'https://api.istex.fr/document/'+istex_id
    queries.append(q)
len(queries)

344

In [51]:
def generate_json_output(queries):
    res = []
    for q in queries:
        response = urllib2.urlopen(q)
        hit = json.load(response)
        missing_abst_count = 0
        article = dict()
        article['doi'] = hit['doi']
        article['istex_id'] = hit['id']
        article['title'] = hit['title']
        try:
            article['abstract'] = hit['abstract']
        except:
            print 'missing abstract of istex_id:', hit['id']
            missing_abst_count = missing_abst_count + 1
            article['abstract'] = ' '
        article['source'] = ['istex']
        #article['host'] = hit['host']
        article['publicationDate'] = hit['publicationDate']
        #article['authors'] = hit['author']
        try:
            article['KeyWords'] = hit['keywords']['teeft']
        except:
            article['KeyWords'] = []
        res.append(article)
    print 'missing abstract count:', missing_abst_count
    return res
queries_results = generate_json_output(queries)
queries_results
json.dump(queries_results, open('initial_corpus_BC.json', 'wb'))

missing abstract of istex_id: 248B68665A4BF1FC3C2A59BC9323988D8ACDE51C
missing abstract count: 0


In [53]:
queries_results[5:7]

[{'KeyWords': [u'tanzania',
   u'usambara',
   u'rewood',
   u'munishi',
   u'biodiversity',
   u'building poles',
   u'tree resources',
   u'ecol',
   u'agroforestry',
   u'catchment',
   u'usambaras',
   u'biodiversity conservation',
   u'blackwell publishing',
   u'basal',
   u'journal compilation',
   u'suppl',
   u'natural forests',
   u'tree species',
   u'firewood',
   u'fodder',
   u'species diversity',
   u'west usambaras',
   u'rewood timber',
   u'west usambara mountains',
   u'farmland',
   u'basal area',
   u'west usambara',
   u'sokoine university',
   u'local people',
   u'tree management',
   u'tree species composition',
   u'diversity index',
   u'montane forests',
   u'forestry',
   u'agroforestry systems',
   u'agricultural crops',
   u'diversity indices',
   u'agricultural landscapes',
   u'species richness',
   u'usambara mountains',
   u'conservation timber',
   u'natural forest',
   u'species composition',
   u'biodiversity management',
   u'dominant species',
  

In [34]:
pickle.dump(initial_corpus, open('initial_corpus_BC.pickle','wb'))

In [25]:
def generate_json_output(queries):
    q = q+'&from='+str(f)
    response = urllib2.urlopen(q)
    data = json.load(response)
    article = dict()
    res = np.array(range(len(data['hits'])), dtype=np.object)
    missing_abst_count = 0
    for (i, hit) in enumerate(data['hits']):
        article = dict()
        article['doi'] = hit['doi']
        article['istex_id'] = hit['id']
        article['title'] = hit['title']
        try:
            article['abstract'] = hit['abstract']
        except:
            print 'missing abstract of istex_id:', hit['id']
            missing_abst_count = missing_abst_count + 1
            article['abstract'] = ' '
        article['source'] = ['istex']
        #article['host'] = hit['host']
        article['publicationDate'] = hit['publicationDate']
        #article['authors'] = hit['author']
        try:
            article['KeyWords'] = hit['keywords']['teeft']
        except:
            article['KeyWords'] = []
        res[i] = article
    print 'page '+str(f)+' missing results count:', missing_abst_count
    return res

def json_query_from(q, f):
    q = q+'&from='+str(f)
    response = urllib2.urlopen(q)
    data = json.load(response)
    article = dict()
    res = np.array(range(len(data['hits'])), dtype=np.object)
    missing_abst_count = 0
    for (i, hit) in enumerate(data['hits']):
        article = dict()
        article['doi'] = hit['doi']
        article['istex_id'] = hit['id']
        article['title'] = hit['title']
        try:
            article['abstract'] = hit['abstract']
        except:
            print 'missing abstract of istex_id:', hit['id']
            missing_abst_count = missing_abst_count + 1
            article['abstract'] = ' '
        article['source'] = ['istex']
        #article['host'] = hit['host']
        article['publicationDate'] = hit['publicationDate']
        #article['authors'] = hit['author']
        try:
            article['KeyWords'] = hit['keywords']['teeft']
        except:
            article['KeyWords'] = []
        res[i] = article
    print 'page '+str(f)+' missing results count:', missing_abst_count
    return res

def json_query(q, file_name):
    response = urllib2.urlopen(q)
    data = json.load(response)
    nb_requests = 1 + data['total'] / 1000
    if nb_requests > 10: # maximum number of pages due to API pagination restrection
        nb_requests = 10
    page_results = json_query_from(q, 0)
    for i in range(nb_requests)[1:]:
        f = i * 1000
        next_request = json_query_from(q, f)
        page_results = np.hstack((page_results, next_request))
    page_results = page_results.tolist()
    json.dump(page_results, open(file_name, 'wb'))
    print 'json file was dumped successfully at;'+file_name
    return page_results

In [27]:
q = 'https://api.istex.fr/document/?q=((subject.value:%22Biodiversity%20Conservation%22%20OR%20keywords:%22Biodiversity%20Conservation%22%20)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id,doi,author,keywords,title,publicationDate,abstract,host'
file_name = 'test_articles.json'
test_json = json_query(q, file_name)

missing abstract of istex_id: 62C727AE41EF251290DD4CCBD614916AC09C818C
missing abstract of istex_id: C717A79D0C721B2D0DD2570E2DC90DDA00A31E36
missing abstract of istex_id: FA39C2494878D3A76117709E0383173AB29128AF
missing abstract of istex_id: 83CE7A03CDCEECADBCDA31AEE54A8065D744AEC1
missing abstract of istex_id: 4CF992BA93D47667B4923333817304DE803141F6
missing abstract of istex_id: FD80A5067300EBCBB5F9A3B252961D77A70F3AD1
missing abstract of istex_id: 292B0D3441B1E3B283066B67C6E79A149D6C6FE4
missing abstract of istex_id: D6480A3B45FDD9F18DBFDC72B98B6CC95745C713
missing abstract of istex_id: 80A196DF94682C904CB47D23778698CFB6ED7986
missing abstract of istex_id: 343318EE74BA1C6E1CE573AB0B501BBAC413A2BA
missing abstract of istex_id: B407EFE7684D5E086829D93ACD3CAED3413DFFC0
missing abstract of istex_id: 87449EC9F085F42CA889920624379CCC8F1BD428
missing abstract of istex_id: 94D846D4602C83206C33A78270904A71385C5A22
missing abstract of istex_id: BA831D079AF57A331E050F1A3DC75DC587E832EC
missin

In [ ]:
q = 'https://api.istex.fr/document/?q=((subject.value:%22Biodiversity%20Conservation%22%20OR%20keywords:%22Biodiversity%20Conservation%22%20)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id,doi,author,keywords,title,publicationDate,abstract,host'
file_name = 'initial__articles.json'
test_json = json_query(q, file_name)

In [10]:
s3h_results = pickle.load(open('results/BC0t_results.pickle', 'rb'))

In [44]:
s3h_syn_results = pickle.load(open('results/BC0t_1_RFC_syn_results.pickle', 'rb'))

In [15]:
len(test)

55

In [34]:
babel_test_intersection = find_intersection(test,babelnet_results)
babel_test_intersection_size = len(babel_test_intersection)

In [38]:
print len (find_intersection(pd.DataFrame(s3h_results.items()[:1000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[1000:2000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[2000:3000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[3000:4000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[4000:5000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[5000:6000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[6000:7000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[7000:8000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[8000:9000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[9000:10000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[10000:20000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[20000:30000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[30000:40000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[40000:50000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[50000:60000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[60000:70000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[70000:80000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[80000:90000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[90000:100000])[0], babelnet_results))

304
213
169
163
137
96
98
103
103
59
531
321
220
190
136
133
130
102
100


In [42]:
print len (find_intersection(pd.DataFrame(s3h_results.items()[100000:200000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[200000:300000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[300000:400000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[400000:500000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[500000:600000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[600000:700000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[700000:800000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[800000:900000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[900000:1000000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[1000000:2000000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[2000000:3000000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[3000000:4000000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[4000000:])[0], babelnet_results))

649
272
221
173
179
182
181
151
177
1356
852
506
45


In [39]:
print len (find_intersection(pd.DataFrame(s3h_results.items()[:1000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:2000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:3000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:4000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:5000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:6000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:7000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:8000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:9000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:10000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:20000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:30000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:40000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:50000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:60000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:70000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:80000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:90000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:100000])[0], babelnet_results))

304
517
686
849
986
1082
1180
1283
1386
1445
1976
2297
2517
2707
2843
2976
3106
3208
3308


In [40]:
print len (find_intersection(pd.DataFrame(s3h_results.items()[:1000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[1000:2000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[2000:3000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[3000:4000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[4000:5000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[5000:6000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[6000:7000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[7000:8000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[8000:9000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[9000:10000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[10000:20000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[20000:30000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[30000:40000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[40000:50000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[50000:60000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[60000:70000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[70000:80000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[80000:90000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[90000:100000])[0], babel_test_intersection))

1
1
1
0
0
0
1
0
0
1
1
3
2
0
0
0
0
0
0


In [43]:
print len (find_intersection(pd.DataFrame(s3h_results.items()[100000:200000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[200000:300000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[300000:400000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[400000:500000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[500000:600000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[600000:700000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[700000:800000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[800000:900000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[900000:1000000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[1000000:2000000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[2000000:3000000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[3000000:4000000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[4000000:])[0], babel_test_intersection))

0
1
0
0
0
0
0
0
0
0
0
0
0


In [41]:
print len (find_intersection(pd.DataFrame(s3h_results.items()[:1000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:2000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:3000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:4000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:5000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:6000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:7000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:8000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:9000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:10000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:20000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:30000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:40000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:50000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:60000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:70000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:80000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:90000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:100000])[0], babel_test_intersection))

1
2
3
3
3
3
4
4
4
5
6
9
11
11
11
11
11
11
11


In [27]:
pd.DataFrame(s3h_results.items()[:1000])

,0,1
0,0A8071F67FEE1A6BF14E5069288C9E67B8A3F2AA,1.000
1,2946E8F7A320DEF1BEFBB26A21CB95E0492FE580,1.000
2,4B1F119CA2211850D0FA755EB5D43D0C26A0358B,1.000
3,396205226F7DDC8B48E8FBEF16684C7703D905D1,1.000
4,FA9D1081F58F45F8168813C84C6D9223BBB1873E,1.000
5,7E4EF8002D4FEA0DE518902EC3D79AE81B29D79A,1.000
6,277C9AC0AF7CB753DBF8609EE57C946AAEF0A172,1.000
7,FAD58615F61C6D0F70C3F054B27D8D7B75AC7C34,1.000
8,D037DE25D3FAE7E3D1F7EC618602687C735224C8,1.000
9,71ADE7014F39A00FDE2D4530BCDD5DF7E509B179,1.000


In [8]:
def eval_babelnet(results, top_n, test):
	top = results[:top_n]
	print 'matched rate at '+str(top_n)+':' , len(find_intersection(test,top))/float(len(test))

In [22]:
print 'BabelNet Method Evaluation:'
for i in range(1000,11000,1000):
    eval_babelnet(babelnet_results, i, test)

BabelNet Method Evaluation:
matched rate at 1000: 0.0181818181818
matched rate at 2000: 0.0181818181818
matched rate at 3000: 0.0363636363636
matched rate at 4000: 0.0727272727273
matched rate at 5000: 0.109090909091
matched rate at 6000: 0.109090909091
matched rate at 7000: 0.145454545455
matched rate at 8000: 0.218181818182
matched rate at 9000: 0.218181818182
matched rate at 10000: 0.218181818182


In [17]:
def eval_use_case(sorted_results, top_n, test):
	top = pd.DataFrame(sorted_results.items()[:top_n])
	print 'matched rate at '+str(top_n)+':' , len(find_intersection(test,top[0]))/float(len(test))

In [18]:
print 'Evaluation:'
for i in range(1000,11000,1000):
    eval_use_case(s3h_results, i, test)

Evaluation:
matched rate at 1000: 0.127272727273
matched rate at 2000: 0.236363636364
matched rate at 3000: 0.290909090909
matched rate at 4000: 0.309090909091
matched rate at 5000: 0.327272727273
matched rate at 6000: 0.345454545455
matched rate at 7000: 0.363636363636
matched rate at 8000: 0.436363636364
matched rate at 9000: 0.436363636364
matched rate at 10000: 0.454545454545


In [45]:
print 'Evaluation Hyprid:'
for i in range(1000,11000,1000):
    eval_use_case(s3h_syn_results, i, test)

Evaluation Hyprid:
matched rate at 1000: 0.109090909091
matched rate at 2000: 0.218181818182
matched rate at 3000: 0.236363636364
matched rate at 4000: 0.327272727273
matched rate at 5000: 0.363636363636
matched rate at 6000: 0.381818181818
matched rate at 7000: 0.381818181818
matched rate at 8000: 0.418181818182
matched rate at 9000: 0.454545454545
matched rate at 10000: 0.472727272727
